In [20]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import cv2
import datetime
import math
import numpy as np
import os
import pathlib
import sklearn.model_selection
import sys
import tensorflow as tf
import time

from albumentations import (
    Compose, HorizontalFlip, ShiftScaleRotate, ElasticTransform,
    RandomBrightness, RandomContrast, RandomGamma
)

from AugmentationSequence import AugmentationSequence
from files import create_folder, save_fit_history, save_lossgraph, save_figs
from metrics import dice_coef, jaccard_distance
from model import evaluate, unet_model, get_loss_function
from save import save

In [21]:
class CreateSequence(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return batch_x, batch_y

# GPU

In [22]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            print(f'GPU: {tf.config.experimental.get_device_details(gpu)["device_name"]}')
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

GPU: NVIDIA GeForce RTX 3060


2022-10-23 17:11:05.539465: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [23]:
cfg = {
    'channel': 1,
    'batch_size': 8,
    'fold': 5,
    'epochs': 75,
    'image_size': 256,
    'learning_rate': 0.001,
    'random_state': 1234,
    'test_size': 0.2,
    'val_size': 0.05,
    'path_dataset': '../dataset',
    'path_out': 'out',
    'loss_function': 'dice',
    'data_augmentation': False
}
images_folder = os.path.join('../dataset_gimp', 'imagens_sp', 'imagens', 'grayscale', 'originais', str(cfg['image_size']), 'jpeg')
masks_folder = os.path.join('../dataset_gimp', 'imagens_sp', 'imagens', 'mask', 'mask_manual', str(cfg['image_size']), 'bmp')

In [24]:
os.path.exists(images_folder), os.path.exists(masks_folder)

(True, True)

In [25]:
list_labels = list([])
list_images = list([])
list_images_names = list([])
def load_files():
    for file in sorted(pathlib.Path(masks_folder).rglob('*')):
        mask = tf.keras.preprocessing.image.load_img(file.resolve(), color_mode='grayscale')
        mask = tf.keras.preprocessing.image.img_to_array(mask)
        mask = mask/255
        list_labels.append(mask)

        if cfg['channel'] == 1:
            image = tf.keras.preprocessing.image.load_img(os.path.join(images_folder, f'{file.stem}.jpeg'), color_mode='grayscale')
        else:
            image = tf.keras.preprocessing.image.load_img(os.path.join(images_folder, f'{file.stem}.jpeg'))
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = image/255
        list_images.append(image)

        list_images_names.append(file)

load_files()
print(len(list_labels), len(list_images), len(list_images_names))

375 375 375


In [26]:
# list_images[0].shape
list_labels[0].shape

(256, 256, 1)

In [27]:
x = np.array(list_images).reshape((len(list_images), cfg['image_size'], cfg['image_size'], cfg['channel']))
y = np.array(list_labels).reshape((len(list_labels), cfg['image_size'], cfg['image_size'], 1))

print(x.shape, y.shape)

(375, 256, 256, 1) (375, 256, 256, 1)


In [28]:
kf = sklearn.model_selection.KFold(n_splits=cfg['fold'], shuffle=True, random_state=cfg['random_state'])

models = []
list_evaluate = list([])
list_time = 0
current_datetime = datetime.datetime.now().strftime('%d-%m-%Y-%H-%M-%S')
path = os.path.join(cfg['path_out'], current_datetime)
create_folder(list([path]))
for fold, (train_index, test_index) in enumerate(kf.split(x)):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(x_train, y_train, test_size=cfg['val_size'], random_state=cfg['random_state'])

    print(x_train.shape)
    print(x_val.shape)
    print(x_test.shape)
    print(x.shape)

    path_fold = os.path.join(path, str(fold))
    create_folder(list([path_fold]))

    augment = Compose([
        HorizontalFlip(),
        ShiftScaleRotate(rotate_limit=45, border_mode=cv2.BORDER_CONSTANT),
        ElasticTransform(border_mode=cv2.BORDER_CONSTANT),
        RandomBrightness(),
        RandomContrast(),
        RandomGamma()
    ])
    steps_per_epoch = math.ceil(x_train.shape[0] / cfg['batch_size'])
    if cfg['data_augmentation']:
        train_generator = AugmentationSequence(x_train, y_train, cfg['batch_size'], augment)
    else:
        train_generator = CreateSequence(x_train, y_train, cfg['batch_size'])
    reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose=1)
    filename_model = os.path.join(path_fold, 'unet.h5')
    checkpointer = tf.keras.callbacks.ModelCheckpoint(filename_model, verbose=1, save_best_only=True)
    strategy = tf.distribute.MirroredStrategy()

    with strategy.scope():
        model = unet_model(cfg)
        adam_opt = tf.keras.optimizers.Adam(learning_rate=cfg['learning_rate'])
        model.compile(optimizer=adam_opt, loss=get_loss_function(cfg['loss_function']), metrics=[dice_coef, jaccard_distance, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    tf.keras.backend.clear_session()
    start_time = time.time()
    fit = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=cfg['epochs'], validation_data=(x_val, y_val), callbacks=[checkpointer, reduce_learning_rate])
    end_time = time.time() - start_time

    save_fit_history(fold, fit, path_fold)
    save_lossgraph(fold, fit, path_fold)
    list_evaluate.append(evaluate(end_time, fold, model, x_train, x_val, x_test, y_train, y_val, y_test))
    list_time += end_time

    models.append(model)

    # model = tensorflow.keras.models.load_model('unet_rgb.h5', custom_objects = {'dice_loss': dice_loss, 'dice_coef': dice_coef, 'jaccard_distance': jaccard_distance })

    save_figs(cfg, list_images_names, test_index, model, path_fold, x)

tf.keras.backend.clear_session()

(177, 256, 256, 1)
(10, 256, 256, 1)
(188, 256, 256, 1)
(375, 256, 256, 1)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


/home/xandao/miniconda3/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1613: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/xandao/miniconda3/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


Epoch 1/8


2022-10-23 17:11:07.422788: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_170920"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:19779"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          siz

12/12 [==============================] - ETA: 0s - loss: 0.5882 - dice_coef: 0.4091 - jaccard_distance: 0.2615 - precision: 0.4150 - recall: 0.8205
Epoch 1: val_loss improved from inf to 0.63905, saving model to out/23-10-2022-17-11-06/0/unet.h5
12/12 [==============================] - 8s 299ms/step - loss: 0.5882 - dice_coef: 0.4091 - jaccard_distance: 0.2615 - precision: 0.4150 - recall: 0.8205 - val_loss: 0.6391 - val_dice_coef: 0.3609 - val_jaccard_distance: 0.2202 - val_precision: 0.0773 - val_recall: 0.0044 - lr: 0.0010
Epoch 2/8
12/12 [==============================] - ETA: 0s - loss: 0.4326 - dice_coef: 0.5570 - jaccard_distance: 0.3890 - precision: 0.5884 - recall: 0.9784
Epoch 2: val_loss improved from 0.63905 to 0.49841, saving model to out/23-10-2022-17-11-06/0/unet.h5
12/12 [==============================] - 2s 164ms/step - loss: 0.4326 - dice_coef: 0.5570 - jaccard_distance: 0.3890 - precision: 0.5884 - recall: 0.9784 - val_loss: 0.4984 - val_dice_coef: 0.5016 - val_jacca

2022-10-23 17:11:47.173700: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_210163"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:24706"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          siz

12/12 [==============================] - ETA: 0s - loss: 0.6644 - dice_coef: 0.3367 - jaccard_distance: 0.2060 - precision: 0.4551 - recall: 0.4101
Epoch 1: val_loss improved from inf to 0.75384, saving model to out/23-10-2022-17-11-06/1/unet.h5
12/12 [==============================] - 7s 205ms/step - loss: 0.6644 - dice_coef: 0.3367 - jaccard_distance: 0.2060 - precision: 0.4551 - recall: 0.4101 - val_loss: 0.7538 - val_dice_coef: 0.2462 - val_jaccard_distance: 0.1404 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 2/8
12/12 [==============================] - ETA: 0s - loss: 0.4840 - dice_coef: 0.5134 - jaccard_distance: 0.3468 - precision: 0.5955 - recall: 0.9307
Epoch 2: val_loss did not improve from 0.75384
12/12 [==============================] - 2s 142ms/step - loss: 0.4840 - dice_coef: 0.5134 - jaccard_distance: 0.3468 - precision: 0.5955 - recall: 0.9307 - val_loss: 0.9995 - val_dice_coef: 5.3794e-04 - val_jaccard_distance: 2.7285e-04 - val_precision: 0.

In [ ]:
save(cfg, sys.argv, images_folder, list_evaluate, list_images, list_labels, list_time, masks_folder, path)